# 역과 세부장소 추천 파일

In [1]:
import pandas as pd
import pickle
import gzip
import os
import pandas as pd
# from dbfread import DBF
import folium
import base64
from folium import plugins
import numpy as np
print(folium.__version__)

0.11.0


In [2]:
#장소랑 지하철 좌표 불러오기
with gzip.open('seoul_populartime.pickle','rb') as f:
    data = pickle.load(f)

df_jwapyo = pd.read_csv('../modeling/data/서울 지하철역 좌표.csv')

In [20]:
subway={}
for i in range(len(df_jwapyo)):
    subway[df_jwapyo['역명'][i]] = [df_jwapyo['위도'][i],df_jwapyo['경도'][i]]

In [21]:
df_jwapyo = df_jwapyo[['역명','경도','위도']].drop_duplicates('역명')
df_jwapyo.rename(columns={'경도':'lng','위도':'lat'}, inplace=True)

# 위도 경도 input으로 받는다, 여기선 예시로 강남역 기준으로 진행

In [22]:
input_lat = subway['강남'][0]
input_lng = subway['강남'][1]
print(input_lat, input_lng)

37.4970466631954 127.02815636174


# 반경 내 역들 listup

In [23]:
upper_right = (input_lat+0.02, input_lng+0.02)
lower_left = (input_lat-0.02, input_lng-0.02)

mask = (
    (df_jwapyo.lat <= upper_right[0]) & (df_jwapyo.lat >= lower_left[0]) &
    (df_jwapyo.lng <= upper_right[1]) & (df_jwapyo.lng >= lower_left[1])
       )

ranged_station = df_jwapyo[mask]['역명']

In [24]:
ranged_station.values.tolist()

['역삼',
 '강남',
 '교대(법원·검찰청)',
 '서초',
 '잠원',
 '남부터미널(예술의전당)',
 '양재(서초구청)',
 '매봉',
 '학동',
 '논현',
 '반포']

# 시간대 입력 (일주일 후까지 가능) 날짜 입력

In [25]:
# input_date, input_time, input_category = input('날짜입력(YYYY-mm-dd) : '), input('시간입력(ex:07 or 20 등) : '), input('카테고리입력 : ')

In [26]:
input_date = '2020-04-05'
input_time = int('19')
input_category = ['맛집','카페/빵집']

# 예측 모델로부터 계산된 파일을 불러온다(여기선 modeling시 사용했던데이터로 테스트)

In [27]:
#하차수 계산
import pandas as pd
df_model_pre = pd.read_pickle('../modeling/merged_exceptsub.pickle')
#범위에 해당하는 역만 filtering
df_model_pre = df_model_pre.query('역명 in {}'.format(ranged_station.values.tolist()))

In [28]:
#input값 시간형식으로 변경
input_date = pd.to_datetime(input_date)
df_model_pre = df_model_pre[['역명','date','시간','하차']][df_model_pre['date']==input_date]
df_model = df_model_pre.groupby(['역명','시간']).sum().unstack()
df_model.columns = df_model.columns.droplevel([0])

In [29]:
#출퇴근시간 붐비는 장소 피하기위해 3000 기준으로 filtering 함
df_model = df_model[((df_model[input_time]<3000)&(df_model[input_time+1]<3000)&(df_model[input_time+2]<3000))]

In [30]:
df_model

시간,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
역명,,,,,,,,,,,,,,,,,,,,
강남,104,452,660,928,1943,1501,1572,1875,2236,2009,1701,1856,2030,1908,1358,1178,889,670,519,0
남부터미널(예술의전당),40,301,276,406,480,487,420,630,727,707,759,846,744,889,753,624,412,231,171,3
논현,12,132,151,272,469,312,256,297,355,323,336,343,382,333,244,261,230,170,111,8
매봉,9,90,53,121,166,136,105,150,161,125,141,161,159,171,136,122,127,111,72,10
반포,12,100,79,140,230,146,123,184,146,134,157,130,143,137,112,78,92,84,53,0
서초,38,188,247,224,389,261,335,293,274,299,211,243,217,175,156,185,146,114,86,0
양재(서초구청),17,370,332,478,686,592,459,586,639,526,511,498,595,704,620,631,476,393,195,24
역삼,55,246,353,454,741,520,428,573,639,603,515,474,467,394,379,364,339,236,183,2
잠원,21,55,43,62,97,50,94,94,115,119,139,153,134,157,126,120,106,89,71,0


# 전년유동감소폭 계산

In [31]:
df_subway = pd.read_pickle('../modeling/data/전년유동감소폭.pickle')
df_subway['year'] = df_subway['date'].dt.year
df_subway['month'] = df_subway['date'].dt.month
df_subway = df_subway.query('month == 5 and 구분 == "하차"')[['역명','year','합 계']].groupby(['역명','year']).sum().unstack()
df_subway.columns = df_subway.columns.droplevel([0])
df_subway['변화량(%)'] = (df_subway[2020]-df_subway[2019])/df_subway[2019]*100

In [32]:
df_model = df_model[[input_time]].merge(df_subway[['변화량(%)']], how='left', on ='역명')

In [33]:
df_model['score'] = -np.log10(df_model[19]) - 0.1*df_model['변화량(%)'] + 5

In [34]:
df_model.sort_values(by=['score'], ascending=False).merge(df_jwapyo, how='left', on='역명')

,역명,19,변화량(%),score,lng,lat
0,잠원,126,-29.601100,5.859739,127.011734,37.513059
1,반포,112,-28.936476,5.844430,127.011795,37.508289
2,서초,156,-28.008078,5.607683,127.010553,37.492788
3,남부터미널(예술의전당),753,-33.559019,5.479107,127.015328,37.486634
4,매봉,136,-25.375438,5.404005,127.046657,37.486984
5,양재(서초구청),620,-28.893310,5.096939,127.033360,37.484769
6,강남,1358,-30.793853,4.946486,127.028156,37.497047
7,학동,168,-20.673188,4.842010,127.031750,37.514291
8,역삼,379,-23.799698,4.801331,127.038307,37.501240
9,논현,244,-19.034889,4.516099,127.021917,37.510292


---
---
# 세부장소 listup과정

In [35]:
# 장소 하나의 데이터프레임으로 만들기
places = list(data.keys())
print(places)
df = pd.DataFrame()
for i in places:
    df1 = pd.DataFrame(data[i])
    df1['places'] = i
    df = pd.concat([df,df1])
df = df.reset_index(drop=True)

['amusement_park', 'aquarium', 'art_gallery', 'bakery', 'bank', 'bar', 'book_store', 'bowling_alley', 'cafe', 'clothing_store', 'department_store', 'doctor', 'drugstore', 'hospital', 'library', 'local_government_office', 'meal_takeaway', 'movie_theater', 'museum', 'night_club', 'park', 'pharmacy', 'post_office', 'restaurant', 'shopping_mall', 'spa', 'store', 'tourist_attraction', 'zoo']


In [36]:
print(len(df),len(df.columns))

21531 12


In [37]:
place_dic = {'amusement_park' : '관광/문화/놀거리' , 'aquarium' : '관광/문화/놀거리' ,
             'art_gallery' : '관광/문화/놀거리' , 'book_store' : '관광/문화/놀거리',
             'bowling_alley' : '관광/문화/놀거리', 'library' : '관광/문화/놀거리',
             'movie_theater' : '관광/문화/놀거리', 'museum' : '관광/문화/놀거리',
             'doctor' : '의료','drugstore' : '의료',
             'hospital' : '의료','pharmacy' : '의료',
             'bar' : '맛집','meal_takeaway' : '맛집','restaurant' : '맛집',
             'bakery' : '카페/빵집', 'cafe' : '카페/빵집',
             'clothing_store' : '쇼핑/마트', 'department_store' : '쇼핑/마트',
             'shopping_mall' : '쇼핑/마트','store' : '쇼핑/마트',
             'bank' : '은행/공공기관', 'local_government_office' : '은행/공공기관','post_office' : '은행/공공기관',
            }

In [38]:
# X, Y 좌표로 데이터 프레임 빼기
def preprocessing_data(df):
    df['lat'] = [df['coordinates'][i]['lat'] for i in range(len(df))]
    df['lng'] = [df['coordinates'][i]['lng'] for i in range(len(df))]
    df['category'] = df['places'].map(place_dic)
    return df
df = preprocessing_data(df)

In [39]:
class DF:
    def __init__(self, df):
        self.df = df
        print('category : 맛집, 카페/빵집, 쇼핑/마트, 은행/공공기관, 관광/문화/놀거리')
        
        
    def listup(self, station, category = 'all', k=0.01):
        input_coordinates = subway[station]
        df['lat_dis'] = abs(df['lat'] - input_coordinates[0])
        df['lng_dis'] = abs(df['lng'] - input_coordinates[1])
        self.df_sta_cat = df.query("lat_dis < {} and lng_dis < {}".format(k,k))
        self.station = station
        if category == 'all':
            return self.df_sta_cat
        else:
            self.df_sta_cat = self.df_sta_cat.query('category in {}'.format(category))
        return self.df_sta_cat

        
    def timeline(self, time = (0,24), week='Monday'):#time=(시작시간,끝시간)
        time_dic={}
        temp = []
        
        for i in self.df_sta_cat['populartimes']:
            for j in i:
                time_dic[j['name']] = j['data']
            temp.append(pd.DataFrame(time_dic)[week][time[0]:time[1]].values)
        self.df_sta_cat['density_of_person'] = temp
        self.df_sta_cat = self.df_sta_cat.reset_index(drop=True)
        self.df_sta_cat = self.df_sta_cat[['name','address','density_of_person','lat','lng','rating', 'rating_n']]
        self.df_sta_cat.density_of_person = [sum(i) for i in self.df_sta_cat.density_of_person]
    def map_list(self):
        self.location = subway[self.station]
        self.m = folium.Map(
            location = self.location,
            zoom_start = 16
        )
      # 이거 특정기준이 잡히면 그냥 하나의 컬럼을 만들어서 color안에 넣을 수 있게 줄일 예정
        t = self.df_sta_cat
        for i in t.index:
            if t['density_of_person'].iloc[i] > 75:
                marker = folium.CircleMarker([t['lat'].iloc[i],t['lng'].iloc[i]],color='red', radius=3)
                marker.add_to(self.m)
            elif t['density_of_person'].iloc[i] > 50:
                marker = folium.CircleMarker([t['lat'].iloc[i],t['lng'].iloc[i]],color='yellow', radius=3)
                marker.add_to(self.m)
            elif t['density_of_person'].iloc[i] > 25:
                marker = folium.CircleMarker([t['lat'].iloc[i],t['lng'].iloc[i]],color='green', radius=3)
                marker.add_to(self.m)
            else:
                marker = folium.CircleMarker([t['lat'].iloc[i],t['lng'].iloc[i]],color='blue', radius=3)
                marker.add_to(self.m)
        return self.m
    
    def  get_geojson_grid(self,n=6):
        all_boxes = []
        upper_right = (self.location[0]+0.005, self.location[1]+0.006)
        lower_left = (self.location[0]-0.005, self.location[1]-0.006)
        
        lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
        lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

        lat_stride = lat_steps[1] - lat_steps[0]
        lon_stride = lon_steps[1] - lon_steps[0]
        
        for lat in lat_steps[:-1]:
                for lon in lon_steps[:-1]:
                    # Define dimensions of box in grid
                    upper_left = [lon, lat + lat_stride]
                    upper_right = [lon + lon_stride, lat + lat_stride]
                    lower_right = [lon + lon_stride, lat]
                    lower_left = [lon, lat]

                    # Define json coordinates for polygon
                    coordinates = [
                        upper_left,
                        upper_right,
                        lower_right,
                        lower_left,
                        upper_left
                    ]

                    geo_json = {"type": "FeatureCollection",
                                "properties":{
                                    "lower_left": lower_left,
                                    "upper_right": upper_right
                                },
                                "features":[]}

                    grid_feature = {
                        "type":"Feature",
                        "geometry":{
                            "type":"Polygon",
                            "coordinates": [coordinates],
                        }
                    }

                    geo_json["features"].append(grid_feature)

                    all_boxes.append(geo_json)
        return all_boxes

# 1. 지역과 카테고리 선택으로 필터링

In [40]:
dd = DF(df)
dd.listup('강남', input_category) ##listup으로 역이랑 카테고리 정함

category : 맛집, 카페/빵집, 쇼핑/마트, 은행/공공기관, 관광/문화/놀거리


,id,name,address,types,coordinates,rating,rating_n,current_popularity,populartimes,time_spent,places,time_wait,lat,lng,category,lat_dis,lng_dis
261,ChIJ7ZTxtUOhfDUR0PEW_uo8vqY,cake artisée(Seocho),"1645 Seocho 1(il)-dong, Seocho-gu","[bakery, food, point_of_interest, store, estab...","{'lat': 37.49176689999999, 'lng': 127.021523}",4.0,119.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[15, 90]",bakery,NaN,37.491767,127.021523,카페/빵집,0.005280,0.006633
262,ChIJ1W-PA0OhfDURIunvyNrtEzM,Hanscake(Seocho),"1635-6 Seocho-dong, Seocho-gu","[bakery, food, point_of_interest, store, estab...","{'lat': 37.493239, 'lng': 127.0215229}",3.9,39.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[10, 45]",bakery,NaN,37.493239,127.021523,카페/빵집,0.003808,0.006633
263,ChIJFYsOVkKhfDURxvTZ_bm19Q4,장티크 (J' antique),"50 Seocho-daero 64-gil, Seocho 1(il)-dong, Seo...","[bakery, restaurant, food, point_of_interest, ...","{'lat': 37.4934232, 'lng': 127.0211658}",4.4,57.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[10, 10]",bakery,NaN,37.493423,127.021166,카페/빵집,0.003623,0.006991
264,ChIJQ_ae7lmhfDURxAWht0gFMio,Krispy Kreme,"지하 396 Gangnam-daero, Yeoksam 1(il)-dong, Gang...","[bakery, food, point_of_interest, store, estab...","{'lat': 37.496551, 'lng': 127.0285717}",3.8,17.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...",NaN,bakery,NaN,37.496551,127.028572,카페/빵집,0.000496,0.000415
265,ChIJMyT_QVqhfDURH3F6sq0D1c0,Dunkin' Donuts,"1328 Seocho-dong, Seocho-gu","[bakery, food, point_of_interest, store, estab...","{'lat': 37.4960102, 'lng': 127.0281623}",4.2,456.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[25, 90]",bakery,NaN,37.496010,127.028162,카페/빵집,0.001036,0.000006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14309,ChIJV3ExelWhfDURlt6iphZiZhA,푸른바다세꼬시,"3 Nonhyeon-ro 81-gil, Gangnam-gu","[restaurant, food, point_of_interest, establis...","{'lat': 37.4980443, 'lng': 127.0376879}",4.2,121.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...",NaN,restaurant,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...",37.498044,127.037688,맛집,0.000998,0.009532
14311,ChIJwx3dEa-jfDURhZZcWSd8yLw,이태리부대찌개,"22 Nonhyeon-ro 94-gil, Gangnam-gu","[restaurant, food, point_of_interest, establis...","{'lat': 37.5025853, 'lng': 127.0381436}",4.1,178.0,20.0,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[45, 45]",restaurant,NaN,37.502585,127.038144,맛집,0.005539,0.009987
14312,ChIJs7Xe8v6jfDUReYxFpkyQe2w,Chicken & Beer,"509 Nonhyeon-ro, Gangnam-gu","[bar, restaurant, food, point_of_interest, est...","{'lat': 37.5018927, 'lng': 127.0360738}",3.9,31.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...",NaN,restaurant,NaN,37.501893,127.036074,맛집,0.004846,0.007917
14313,ChIJreXTjtGjfDUR52Uor3Fg17A,양궈푸 마라탕,"670-22 Yeoksam-dong, Gangnam-gu","[restaurant, food, point_of_interest, establis...","{'lat': 37.5027904, 'lng': 127.0379838}",4.2,22.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[45, 45]",restaurant,NaN,37.502790,127.037984,맛집,0.005744,0.009827


# 2.시간대, 요일로 필터링

In [41]:
dd.timeline(time=(22,23), week='Friday') ##특정시간대 인구수 표시

In [42]:
dd.df_sta_cat

,name,address,density_of_person,lat,lng,rating,rating_n
0,cake artisée(Seocho),"1645 Seocho 1(il)-dong, Seocho-gu",0,37.491767,127.021523,4.0,119.0
1,Hanscake(Seocho),"1635-6 Seocho-dong, Seocho-gu",0,37.493239,127.021523,3.9,39.0
2,장티크 (J' antique),"50 Seocho-daero 64-gil, Seocho 1(il)-dong, Seo...",0,37.493423,127.021166,4.4,57.0
3,Krispy Kreme,"지하 396 Gangnam-daero, Yeoksam 1(il)-dong, Gang...",0,37.496551,127.028572,3.8,17.0
4,Dunkin' Donuts,"1328 Seocho-dong, Seocho-gu",31,37.496010,127.028162,4.2,456.0
...,...,...,...,...,...,...,...
561,푸른바다세꼬시,"3 Nonhyeon-ro 81-gil, Gangnam-gu",0,37.498044,127.037688,4.2,121.0
562,이태리부대찌개,"22 Nonhyeon-ro 94-gil, Gangnam-gu",0,37.502585,127.038144,4.1,178.0
563,Chicken & Beer,"509 Nonhyeon-ro, Gangnam-gu",56,37.501893,127.036074,3.9,31.0
564,양궈푸 마라탕,"670-22 Yeoksam-dong, Gangnam-gu",0,37.502790,127.037984,4.2,22.0


In [43]:
dd.map_list()

# 3.주변구역을 격자화 및 최적구역 선정

In [44]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import json
import copy
import branca
grid = dd.get_geojson_grid(6)
ranged_data = dd.df_sta_cat
m  =dd.m
# Calculate exposures in grid
popups = []
score_counts = []
top3region=[]
score=0
for i, box in enumerate(grid):
    
    upper_right = box["properties"]["upper_right"]
    lower_left = box["properties"]["lower_left"]

    mask = (
        (ranged_data.lat <= upper_right[1]) & (ranged_data.lat >= lower_left[1]) &
        (ranged_data.lng <= upper_right[0]) & (ranged_data.lng >= lower_left[0])
           )

    num_store = len(ranged_data[mask])+2
    density_per_store = ranged_data[mask].density_of_person.sum()/num_store + 5
    score = num_store/density_per_store
    score_counts.append(score)
    
    if i<3:
        top3region.append((lower_left,upper_right, score))
    else:
        top3region.sort(key=lambda x:x[2])
        
        if top3region[0][2] <score:
            top3region.pop(0)
            top3region.append((lower_left,upper_right, score))
    
    content = "가게 수 {:,.1f}, 혼잡도점수 {:,.1f}".format(num_store, score)
    popup = folium.Popup(content)
    popups.append(popup)
    

best_region = max(score_counts)

# Add GeoJson to map
for i, box in enumerate(grid):
    geo_json = json.dumps(box)

    color = plt.cm.RdBu(score_counts[i] / best_region)
    color = mpl.colors.to_hex(color)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.5,
                                                                    })

    gj.add_child(popups[i])
    m.add_child(gj)
    colormap = branca.colormap.linear.RdBu_09.scale(0, 1)
    colormap = colormap.to_step(index=[0, 0.3, 0.6, 0.8 , 1])
    colormap.caption = 'Relative density score per cell'
colormap.add_to(m)
# Create popups
popup_content = []
# for incident in accident_data.itertuples():
#     number_of_vehicles = "Number of vehicles: {} ".format(incident.Number_of_Vehicles)
#     number_of_casualties = "Number of casualties: {}".format(incident.Number_of_Casualties)
#     content = number_of_vehicles + number_of_casualties
#     popup_content.append(content)

# popups = [folium.Popup(content) for content in popup_content]

# cluster = MarkerCluster(locations=locations, icons=icons, popups=popups)
m#.add_child(cluster)

# 4. 선정된 구역 내 가게들을 혼잡도, 평점, 리뷰수 등을 바탕으로 scoring

In [45]:
listup = pd.DataFrame()
for i, region in enumerate(top3region):
    
    upper_right = region[1]
    lower_left = region[0]

    mask = (
        (ranged_data.lat <= upper_right[1]) & (ranged_data.lat >= lower_left[1]) &
        (ranged_data.lng <= upper_right[0]) & (ranged_data.lng >= lower_left[0])
           )

    listup=pd.concat([listup,ranged_data[mask]])
    
listup.rating_n.clip(0,300)#300이상은 300으로 통일
listup['scoring'] = listup.rating - listup.density_of_person*0.01 + np.log10(listup.rating_n)/3
listup

,name,address,density_of_person,lat,lng,rating,rating_n,scoring
105,바나프레소 국기원사거리점,"827-48 Yeoksam-dong, Gangnam-gu",0,37.497156,127.031706,4.3,65.0,4.904304
106,TOUS les JOURS,"21 Teheran-ro 8-gil, Gangnam-gu",13,37.497576,127.032007,4.5,11.0,4.717131
159,비스트플래닛 BeastPlanet,"B1, 크레신타워 2차, 7 Teheran-ro 8-gil, Gangnam-gu",0,37.498399,127.031688,5.0,3.0,5.159040
160,Angel-in-us Coffee,"116 Teheran-ro, Yeoksam-dong, Gangnam-gu",89,37.498443,127.030521,3.8,191.0,3.670344
262,Sumijeong,"33 Gangnam-daero 84-gil, Gangnam-gu",0,37.497516,127.030939,4.0,6.0,4.259384
265,고등어식당,KR 서울특별시 강남구 강남대로84길 33 대우디오빌플러스 B1층,0,37.497516,127.031077,4.4,16.0,4.801373
266,서진식당,"33 Gangnam-daero 84-gil, Gangnam-gu",0,37.497382,127.031007,4.2,122.0,4.895453
316,통돼지찌개집,"34 Gangnam-daero 84-gil, Gangnam-gu",20,37.497056,127.031311,3.8,44.0,4.147818
317,풍국면 강남직영점,"1층, 화인강남빌딩, KR 서울특별시 강남구 823-14, 21 Teheran-ro...",0,37.497588,127.032000,4.3,414.0,5.172333
350,반미홍호아,"1층, 한라클래식오피스텔, 824-11번지 Yeoksam-dong, Gangnam-gu",0,37.497092,127.030372,3.6,12.0,3.959727


# 5. 최종 top5 listup

In [46]:
top5 = listup.sort_values(by=['scoring'], ascending=False)[:5]
top5

,name,address,density_of_person,lat,lng,rating,rating_n,scoring
93,O'sulloc Green Tea,"373 Gangnam-daero, Seocho-dong, Seocho-gu",0,37.495922,127.028060,4.3,880.0,5.281494
298,Cuchara,"11 Seocho-daero 74-gil, Seocho-gu",0,37.496545,127.026975,4.3,622.0,5.231263
317,풍국면 강남직영점,"1층, 화인강남빌딩, KR 서울특별시 강남구 823-14, 21 Teheran-ro...",0,37.497588,127.032000,4.3,414.0,5.172333
159,비스트플래닛 BeastPlanet,"B1, 크레신타워 2차, 7 Teheran-ro 8-gil, Gangnam-gu",0,37.498399,127.031688,5.0,3.0,5.159040
538,육전식당 (강남점),"11-4 Teheran-ro 8-gil, Gangnam-gu",26,37.497988,127.031996,4.3,509.0,4.942239


In [47]:
top5[['name','density_of_person','rating','rating_n','scoring']]

,name,density_of_person,rating,rating_n,scoring
93,O'sulloc Green Tea,0,4.3,880.0,5.281494
298,Cuchara,0,4.3,622.0,5.231263
317,풍국면 강남직영점,0,4.3,414.0,5.172333
159,비스트플래닛 BeastPlanet,0,5.0,3.0,5.159040
538,육전식당 (강남점),26,4.3,509.0,4.942239


# visulization

In [48]:
for idx, i in top5.iterrows():
    popup = i['name']
    marker = folium.Marker([i['lat'], i['lng']], popup=popup, icon = folium.Icon(icon='star',color = 'red'))

    m.add_child(marker)
# m.save('test.html')

In [49]:
m